In [ ]:
%load_ext autoreload
%autoreload 2

import copy

import numpy as np
import pandas as pd
import scipy.stats as st

import re #regex

import cmdstanpy
import arviz as az

import bebi103
import bokeh_catplot

import bokeh.io
import bokeh.plotting
# import bokeh.models.mappers
import bokeh.palettes

import holoviews as hv
import holoviews.operation.datashader
hv.extension('bokeh')
bebi103.hv.set_defaults()

import panel as pn
pn.extension()

bokeh.io.output_notebook()


%load_ext blackcellmagic

### Munging
Load data from Brewster, pre-tidied by Manuel, and drop the spurious column that was the index in csv.
See `code/exploratory/fish_munging.ipynb` for details. TL;DR: don't use the regulated csv, the one below has all the FISH data. mRNA_cell is the data we want, not spots_totals (some of the repressed strains have higher spots_totals than UV5, so that's clearly not the readout we want).

In [ ]:
df_fish = pd.read_csv("../../data/jones_brewster_2014.csv")
del df_fish['Unnamed: 0']
df_fish.head()

Next, let's get the energies from the supplement of Brewster/Jones 2012 paper.

In [ ]:
df_energies = pd.read_csv("../../data/brewster_jones_2012.csv")
df_energies.head()

All the promoters in the 2012 dataset are in the 2014 fish dataset (verified in `code/exploratory/fish_munging.ipynb`). These are the only constitutive promoters I'm interested in (this only excludes a couple, and they are useless without more metadata).

#### Splitting into regulated & constitutive data
Some of these datasets are not of interest right now so let's split it into multiple dataframes for easier downstream handling. The regulated datasets start with O1, O2, or O3. Everything else doesn't. From that everything else, grab the ones that we have energies for, and set aside the rest. Use regex to parse.

In [ ]:
raw_expt_labels = df_fish['experiment'].unique()
raw_expt_labels.sort()

# put all strings that start w/ 'O' in one list
regulated_labels = [label for label in raw_expt_labels if re.match('^O', label)]
# from that, split out those we have energies for
constitutive_labels = [label for label in raw_expt_labels if label in tuple(df_energies.Name)]

Without more metadata, I don't really know what to do with the leftover labels data, e.g., what good does the aTc concentration do me if I don't know what promoter it was for?

Now that we've got labels we want, let's slice dataframes accordingly.

In [ ]:
df_reg = df_fish[df_fish['experiment'].isin(regulated_labels)]
df_unreg = df_fish[df_fish['experiment'].isin(constitutive_labels)]

Also separate UV5 for testing convenience.

In [ ]:
df_UV5 = df_unreg[df_unreg["experiment"] == "UV5"]

## Analyzing constitutive UV5 with Stan

Stan model borrows from JB's tutorial 7a, 2018. Stan parametrizes the negative binomial with $\alpha$ and $\beta$, where $\alpha$ is the burst frequency (dimensionless, nondimensionalized by mRNA lifetime) and $\beta = 1/b$ where $b$ is the mean burst size.

### Prior predictive checks

In [ ]:
sm_prior_predictive = cmdstanpy.CmdStanModel(
    stan_file="stan/constitutive_prior_predictive_v01.stan"
)
print(sm_prior_predictive.code())

In [ ]:
data_prior_pred = dict(
    N=len(df_UV5), 
    log_alpha_loc=0.5, 
    log_alpha_scale=2.0, 
    log_b_loc=0.5,
    log_b_scale=0.5
)

In [ ]:
prior_pred_samples = sm_prior_predictive.sample(
    data=data_prior_pred,
    fixed_param=True,
    sampling_iters=1000,
    output_dir="./stan/stan_samples",
)

In [ ]:
# Convert to ArviZ InferenceData
prior_pred_samples = az.from_cmdstanpy(
    posterior=prior_pred_samples,
    prior=prior_pred_samples,
    prior_predictive=['mRNA_counts']
)

In [ ]:
p = bebi103.viz.predictive_ecdf(
    prior_pred_samples.prior_predictive['mRNA_counts'],
    frame_height=250,
    frame_width=350,
    discrete=True,
    x_axis_label='mRNA counts',
#     x_axis_type='log'
)
bokeh.io.show(p)

### Simulation-based calibration

Next up: simulation-based calibration (SBC). Quoting JB, this checks "that the sampler can effectively sample the entire space of parameters covered by the prior." We'll go ahead and set up the data for the posterior, even though we won't be sampling the posterior right now. Then we can set up the model.

In [ ]:
data_UV5 = copy.deepcopy(data_prior_pred)
data_UV5["N"] = len(df_UV5)
data_UV5["mRNA_counts"] = df_UV5["mRNA_cell"].values.astype(int)
data_UV5["ppc"] = 0

sm = cmdstanpy.CmdStanModel(stan_file="stan/constitutive_v01.stan")
print(sm.code())

In [ ]:
try:
    sbc_output = pd.read_csv("stan/sbc_output_v01.csv")
except:
    sbc_output = bebi103.stan.sbc(
        prior_predictive_model=sm_prior_predictive,
        posterior_model=sm,
        prior_predictive_model_data=data_prior_pred,
        posterior_model_data=data_UV5,
        measured_data=["mRNA_counts"],
        parameters=["alpha", "b"],
        sampling_kwargs={'thin': 10},
        cores=7,
        N=400,
        progress_bar=True,
    )

    sbc_output.to_csv("stan/sbc_output_v01.csv", index=False)

Plot ECDFs of the rank statistics, which should be ~ uniform. Color according to warning code, which will also let us know if there were any issues with divergences, R-hat, EBFMI, effective number of steps, or tree depth.

In [ ]:
plots = [
    bokeh_catplot.ecdf(
        data=sbc_output.loc[sbc_output["parameter"] == param, :],
        val="rank_statistic",
        cats="warning_code",
        kind="colored",
        frame_width=400,
        frame_height=150,
        title=param,
        conf_int=True,
    )
    for param in sbc_output["parameter"].unique()
]

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=1))

Very uniform. Warning code 2 means R-hat failure. Let's look at the R-hat values.

In [ ]:
bokeh.io.show(
    bokeh_catplot.ecdf(data=sbc_output, val="Rhat", cats="parameter")
)

Not terrible - about 15% are above the brightline 1.01, but not way over. JB suggests in this case that more samples from the posterior might be wise.

JB also has a nice helper function to plot the difference from uniform of rank ECDFs, with a confidence interval for the uniform distribution to guide the eye. Let's take a look.

In [ ]:
bokeh.io.show(bebi103.viz.sbc_rank_ecdf(sbc_output))

Finally let's check shrinkage and z-scores.

In [ ]:
hv.Points(
    data=sbc_output,
    kdims=['shrinkage', 'z_score'],
    vdims=['parameter', 'ground_truth', 'mean', 'sd'],
).opts(
    color='parameter',
    alpha=0.3,
    xlim=(0, 1.05),
    tools=['hover']
)

z-scores are great. There are a couple of points with terrible shrinkage that may be worth investigating. Generally $\alpha$ shrinkage is great, $b$ less so. This may just reflect that my prior on $b$ was pretty tight; less than ~5% have shrinkage less than 0.9, and $\gtrsim 90\%$ have shrinkage above 0.98. I wonder too if the outliers are datasets that just had very very few mRNAs, which would make it hard to inform the posterior. Most of the $b$ points with poor shrinkage have $b >1$, which, if they also had very small $\alpha$, is not inconsistent with my above guess. I should probably relax my priors on $\alpha$ and especially $b$, now that I realized my mistake in plotting the prior predictives (forgot the `discrete` option).

### Sampling the Posterior
We already finished building the model in order to do SBC. Now we just run it.

In [ ]:
# We do want posterior predictive checks this time
data_UV5["ppc"] = 1

posterior_samples = sm.sample(data=data_UV5, cores=6)
posterior_samples = az.from_cmdstanpy(
    posterior_samples, posterior_predictive=["mRNA_counts_ppc"]
)

In [ ]:
bebi103.stan.check_all_diagnostics(posterior_samples)

Good, no sampler warnings. Let's visualize the posterior.

In [ ]:
bokeh.io.show(
    bebi103.viz.corner(
        posterior_samples,
        pars=["alpha", "b"],
        alpha=0.1,
        xtick_label_orientation=np.pi / 4,
    )
)

That looks quite reasonable. The transcripts per burst & burst frequency are both comparable to what we would have inferred from Manuel's MCMC, but now both parameters are actually identifiable!

### Posterior predictive checks

Even though the posterior looks ok, the model could still be wrong: it is identifiable, but is it consistent with the data? Posterior predictive checks address this by asking whether the model could plausibly generate the observed data. (Function borrowed from JB's finch beak tutorial for bebi103b 2020 TAs.)

In [ ]:
def ppc_ecdfs(posterior_samples, df):
    """Plot posterior predictive ECDFs."""
    n_samples = (
        posterior_samples.posterior_predictive.dims["chain"]
        * posterior_samples.posterior_predictive.dims["draw"]
    )

    p1 = bebi103.viz.predictive_ecdf(
        posterior_samples.posterior_predictive["mRNA_counts_ppc"].values.reshape(
            (n_samples, len(df))
        ),
        data=df["mRNA_cell"],
        discrete=True,
        x_axis_label="mRNA counts per cells",
        frame_width=200,
        frame_height=200
    )

    p2 = bebi103.viz.predictive_ecdf(
        posterior_samples.posterior_predictive["mRNA_counts_ppc"].values.reshape(
            (n_samples, len(df))
        ),
        data=df["mRNA_cell"],
        percentiles=[95, 90, 80, 50],
        diff=True,
        discrete=True,
        x_axis_label="mRNA counts per cells",
        frame_width=200,
        frame_height=200
    )
    p1.x_range = p2.x_range
    
    return [p1, p2]

bokeh.io.show(bokeh.layouts.gridplot(ppc_ecdfs(posterior_samples, df_UV5), ncols=2))

The model is not obviously wrong, but depending what percentiles I choose to plot, I might say that a few too many of the observed datapoints are outside the posterior predictive bands.

Still, we should be systematic and plot the posterior and ppc for all the promoters.

### Sampling all the data
Let's repeat for all the constitutive promoters! Since we have so many, do separate loops to generate the samples and generate viz (so we can tweak viz without pointlessly rerunning the sampling).

In [ ]:
# suppress many line of cmdstanpy output, uncomment if you want to watch it sample
%%capture
all_samples = {}
for gene in df_unreg['experiment'].unique():
    temp_df = df_unreg[df_unreg['experiment'] == gene]
    data = copy.deepcopy(data_prior_pred)
    data["N"] = len(temp_df)
    data["mRNA_counts"] = temp_df["mRNA_cell"].values.astype(int)
    data["ppc"] = 1
    
    posterior_samples = sm.sample(data=data, cores=6)
    all_samples[gene] = az.from_cmdstanpy(
        posterior_samples, posterior_predictive=["mRNA_counts_ppc"]
    )

It's always wise to check diagnostics.

In [ ]:
for gene in all_samples:
    print(gene)
    bebi103.stan.check_all_diagnostics(all_samples[gene])

There's some run-to-run variation. Sometimes one or two promoters give Rhat warnings a bit over 1.01, but generally this looks ok.

Now that we've sampled we can plot. (I can't figure out how to add titles to the bokeh layouts that `viz.corner` returns, so I'm doing the poor man's version for now.)

In [ ]:
for gene in all_samples:
    print(gene)
    # plot posterior as corner plot
    bokeh.io.show(
        bebi103.viz.corner(
            all_samples[gene],
            pars=["alpha", "b"],
            alpha=0.1,
            xtick_label_orientation=np.pi / 4,
        )
    )
    # plot post pred ecdf
    obs_data = df_unreg[df_unreg["experiment"] == gene]
    bokeh.io.show(
        bokeh.layouts.gridplot(ppc_ecdfs(all_samples[gene], obs_data), ncols=2)
    )

The posterior predictive checks make it pretty clear that this most simple of models is not entirely capable of producing the observed data for all the promoters. The observed data has a pretty stereotyped pattern on the difference-in-ECDF plots. This strongly suggests there is something systematic our model fails to capture. Nevertheless, looking at the full ecdfs, I'm actually pleasantly surprised that the model does as well as it does. We haven't explicitly put in anything like cell cycle & gene copy number variability, and yet it captures quite a lot. Maybe it's already robust enough to attempt inference on simple repression data?

### Correlation with predicted promoter binding energies

We would like to plot all 18 promoters together to compare their inferred burst parameters at a glance. Plotting all the posterior samples would be visually illegible.
Instead we can compute a contour that encloses 95% of the samples. (Default smoothing in the contour calculator occasionally breaks and totally misses the HPD, so I increased it slightly.) Further, we can color code the contours by the predicted binding energies from Brewster/Jones 2012.

First compute all the contours coords with JB's utility. (Note that `hv.Contours` prefers its input as a dictionary: `"x"` and `"y"` keys must be labeled as such and provide the coords of the contour, and a 3rd key providing a scalar for the contour level, here the promoter binding energy which we lookup.) Then we can overlay all the contours, colored by their corresponding promoter binding energy (fat lines make it easier to perceive the color).

In [ ]:
contour_list = []
for gene in all_samples:
    alpha_samples = all_samples[gene].posterior.alpha.values.flatten()
    b_samples = all_samples[gene].posterior.b.values.flatten()
    x_contour, y_contour = bebi103.viz.contour_lines_from_samples(
        alpha_samples, b_samples, levels=0.95, smooth=0.025
    )
    contour_list.append(
        {
            "x": x_contour[0],
            "y": y_contour[0],
            "Energy (kT)": df_energies.loc[
                df_energies["Name"] == gene, "Energy (kT)"
            ].values[0],
            "Promoter": gene
        }
    )
p = (
    hv.Contours(contour_list, vdims=["Energy (kT)", "Promoter"])
    .opts(logx=True, logy=True)
    .opts(line_width=2)
)
p.opts(
    hv.opts.Contours(
        cmap="viridis",
        colorbar=True,
        tools=["hover"],
        width=500,
        height=500,
        xlabel="α (bursts per mRNA lifetime)",
        ylabel="b (transcripts per burst)",
        padding=0.03,
    )
)

Very interesting. By eye I'd say there's little to no correlation between burst size $b$ and binding energy, and maybe linear scaling between the log of burst frequency $\alpha$ and binding energy, though that correlation is quite noisy. Not immediately obvious how to proceed from a theory/modeling viewpoint.